In [1]:
import networkx as nx
import pandas as pd
import itertools

# 读取指定的工作表
df2 = pd.read_excel('(申请号去重后）美国和欧盟 已筛选TAC_(poly butylene adipate-co-terephthalate) OR TAC_(PBAT).xlsx', sheet_name='删除金发后的专利权人共现')

inventors = []
for i, row in df2.iterrows():
    names = row["[标]原始申请(专利权)人（外文）"].split(" | ")
    inventors.append(names)

# 创建图形对象
G = nx.Graph()

# 添加节点和边，并加权
for i in inventors:
    if len(i) == 1:
        # 如果只有一个专利权人，将其作为单独的节点添加到图形中
        G.add_node(i[0])
    else:
        G.add_nodes_from(i)
        edges = itertools.combinations(i, 2)
        for edge in edges:
            if G.has_edge(*edge):
                # 如果边已经存在，增加权重
                G[edge[0]][edge[1]]['weight'] += 1
            else:
                # 如果边不存在，设置初始权重为1
                G.add_edge(edge[0], edge[1], weight=1)
# 计算节点之间的距离
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
# 将带权邻接矩阵保存为 CSV 文件
adjacency_matrix = nx.to_pandas_adjacency(G, weight='weight')
adjacency_matrix.to_csv('weighted_adjacency_matrix.csv', sep=';')

print("带权邻接矩阵已保存为 weighted_adjacency_matrix.csv 文件。")

带权邻接矩阵已保存为 weighted_adjacency_matrix.csv 文件。


In [2]:
# 计算联系数（边的总数）
number_of_ties = G.number_of_edges()

# 计算网络密度
network_density = nx.density(G)

# 计算网络中心化程度
network_centralization = nx.degree_centrality(G)
max_degree = max(network_centralization.values())
network_centralization = sum([(max_degree - degree) / (len(G)-1) for degree in network_centralization.values()])

print("Number of ties:", number_of_ties)
print("Network density:", network_density)
print("Network centralization:", network_centralization)

Number of ties: 87
Network density: 0.028237585199610515
Network centralization: 0.0363247863247863


In [9]:
# 计算节点的度
degrees = dict(G.degree())

# 计算节点之间的距离
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))

# 计算基于距离的凝聚性
total_distance = sum(sum(length for length in paths.values()) for paths in shortest_paths.values())
distance_based_cohesion = total_distance / (len(G)*(len(G)-1))

# print("Degrees:", degrees)
print("Distance-based cohesion:", distance_based_cohesion)

Distance-based cohesion: 0.03213242453748783


In [10]:
def distance_based_agglomeration(G):
    # 初始化凝聚性度量
    total_agglomeration = 0
    
    # 获取图中所有节点对之间的最短路径长度
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    
    # 遍历图中的每一对节点
    for u in G.nodes():
        for v in G.nodes():
            if u != v:  # 排除自身节点
                # 获取节点u和v之间的最短路径长度
                if v in shortest_paths[u]:
                    distance_uv = shortest_paths[u][v]
                else:
                    # 如果不存在路径，则将距离设置为无穷大
                    distance_uv = float('inf')
                
                # 计算节点之间的凝聚性
                agglomeration_uv = 1 / (1 + distance_uv)
                
                # 更新总的凝聚性度量
                total_agglomeration += agglomeration_uv
    
    # 返回平均凝聚性
    num_nodes = len(G.nodes())
    return total_agglomeration / (num_nodes * (num_nodes - 1))

# 计算基于距离的凝聚性
agglomeration = distance_based_agglomeration(G)
print("基于距离的凝聚性:", agglomeration)

基于距离的凝聚性: 0.014767932489451477
